In [36]:
import pandas as pd
prices = pd.read_csv("all_stocks.csv")
prices.rename(columns={"Date":"ds", "Ticker":"unique_id", "Close":"y"}, inplace=True)
prices["ds"] = prices["ds"].str[:10] # remove time
prices["ds"] = pd.to_datetime(prices["ds"])
prices = prices[["ds", "y", "unique_id","12-day EMA", "26-day EMA", "MACD", "Signal Line", "MACD Histogram", "RSI", "OBV"]]
prices.dropna(inplace=True)

In [37]:
prices

,ds,y,unique_id,12-day EMA,26-day EMA,MACD,Signal Line,MACD Histogram,RSI,OBV
13,2015-03-11,27.299576,AAPL,28.356989,28.633862,-0.276874,-0.114716,-0.162158,32.531122,-8.955360e+07
14,2015-03-12,27.793125,AAPL,28.270240,28.571585,-0.301345,-0.152042,-0.149303,39.016817,1.038972e+08
15,2015-03-13,27.601067,AAPL,28.167291,28.499695,-0.332404,-0.188114,-0.144290,26.879368,-1.034120e+08
16,2015-03-16,27.904793,AAPL,28.126906,28.455628,-0.328722,-0.216236,-0.112486,32.710562,4.008520e+07
17,2015-03-17,28.371550,AAPL,28.164544,28.449400,-0.284856,-0.229960,-0.054896,45.533373,2.441776e+08
...,...,...,...,...,...,...,...,...,...,...
12570,2025-02-12,131.139999,NVDA,129.360865,131.287485,-1.926620,-2.795096,0.868476,38.977245,1.056302e+11
12571,2025-02-13,135.289993,NVDA,130.273038,131.583967,-1.310929,-2.498262,1.187334,44.944130,1.058277e+11
12572,2025-02-14,138.850006,NVDA,131.592572,132.122192,-0.529620,-2.104534,1.574914,69.701054,1.060231e+11
12573,2025-02-18,139.399994,NVDA,132.793714,132.661289,0.132425,-1.657142,1.789567,62.443204,1.062423e+11


In [38]:
# Train / Test split
train_size = 0.8  # 80% train, 20% test
def train_test_split(group):
    split_index = int(len(group) * train_size)
    return group.iloc[:split_index], group.iloc[split_index:]

# Apply the split to each unique stock (grouped by 'unique_id')
train_data, test_data = zip(*prices.groupby("unique_id").apply(train_test_split))

# Convert tuple results to DataFrames
df_train = pd.concat(train_data).reset_index(drop=True)
df_test = pd.concat(test_data).reset_index(drop=True)

/var/folders/vw/fk4p6s510g7dfnhknk592c7r0000gn/T/ipykernel_81656/359759405.py:8: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  train_data, test_data = zip(*prices.groupby("unique_id").apply(train_test_split))


In [39]:
df_train

,ds,y,unique_id,12-day EMA,26-day EMA,MACD,Signal Line,MACD Histogram,RSI,OBV
0,2015-03-11,27.299576,AAPL,28.356989,28.633862,-0.276874,-0.114716,-0.162158,32.531122,-8.955360e+07
1,2015-03-12,27.793125,AAPL,28.270240,28.571585,-0.301345,-0.152042,-0.149303,39.016817,1.038972e+08
2,2015-03-13,27.601067,AAPL,28.167291,28.499695,-0.332404,-0.188114,-0.144290,26.879368,-1.034120e+08
3,2015-03-16,27.904793,AAPL,28.126906,28.455628,-0.328722,-0.216236,-0.112486,32.710562,4.008520e+07
4,2015-03-17,28.371550,AAPL,28.164544,28.449400,-0.284856,-0.229960,-0.054896,45.533373,2.441776e+08
...,...,...,...,...,...,...,...,...,...,...
10000,2023-02-13,194.639999,TSLA,186.361009,170.243320,16.117689,12.806343,3.311347,75.737905,1.401762e+10
10001,2023-02-14,209.250000,TSLA,189.882392,173.132704,16.749689,13.595012,3.154677,78.767977,1.423407e+10
10002,2023-02-15,214.240005,TSLA,193.629717,176.177689,17.452028,14.366415,3.085613,76.505256,1.441508e+10
10003,2023-02-16,202.039993,TSLA,194.923606,178.093415,16.830191,14.859170,1.971021,62.523342,1.418549e+10


In [40]:
df_test

,ds,y,unique_id,12-day EMA,26-day EMA,MACD,Signal Line,MACD Histogram,RSI,OBV
0,2023-02-21,146.997559,AAPL,149.455721,145.805924,3.649796,4.025091,-0.375294,56.786658,8.667857e+08
1,2023-02-22,147.423279,AAPL,149.143038,145.925728,3.217309,3.863534,-0.646225,55.823627,9.177970e+08
2,2023-02-23,147.908371,AAPL,148.953089,146.072591,2.880498,3.666927,-0.786429,47.787116,9.661912e+08
3,2023-02-24,145.245255,AAPL,148.382653,146.011307,2.371346,3.407811,-1.036465,35.419247,9.107216e+08
4,2023-02-27,146.443146,AAPL,148.084267,146.043295,2.040972,3.134443,-1.093471,42.651527,9.557201e+08
...,...,...,...,...,...,...,...,...,...,...
2500,2025-02-12,336.510010,TSLA,368.456215,383.793063,-15.336848,-7.418631,-7.918216,23.253195,1.707751e+10
2501,2025-02-13,355.940002,TSLA,366.530644,381.729873,-15.199229,-8.974751,-6.224478,33.712856,1.716695e+10
2502,2025-02-14,355.839996,TSLA,364.885929,379.812105,-14.926176,-10.165036,-4.761140,35.865327,1.709867e+10
2503,2025-02-18,354.109985,TSLA,363.228092,377.908244,-14.680152,-11.068059,-3.612093,35.032668,1.704704e+10


In [41]:
from neuralforecast import NeuralForecast
from neuralforecast.models import LSTM

input_sizes = [10,15,30,60,90] #historical window
#input_sizes = [10]

for interval in input_sizes:
    lstm = LSTM(h=100,
                    input_size=interval,
                    scaler_type='standard',
                    encoder_hidden_size=128,
                    decoder_layers=2,
                    decoder_hidden_size=128)

    nf = NeuralForecast(
        models=[lstm],
        freq='D'
    )

    print(f"Training for interval: {interval}")
    nf.fit(df=df_train)
    
    #test_df_future = nf.make_future_dataframe(df_train)
    forecasts = nf.predict(df_train)
    print(f"Reults for interval: {interval}")
    print(forecasts)
    test_results = pd.merge(df_test, forecasts, on=["ds", "unique_id"], how="inner") #LSTM (=predicted), y (=actual)
    test_results.to_csv(f"res_lstm_{interval}.csv")


Seed set to 1
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs

  | Name            | Type          | Params | Mode 
----------------------------------------------------------
0 | loss            | MAE           | 0      | train
1 | padder          | ConstantPad1d | 0      | train
2 | scaler          | TemporalNorm  | 0      | train
3 | hist_encoder    | LSTM          | 199 K  | train
4 | context_adapter | Linear        | 129 K  | train
5 | mlp_decoder     | MLP           | 1.5 K  | train
----------------------------------------------------------
329 K     Trainable params
0         Non-trainable params
329 K     Total params
1.319     Total estimated model params size (MB)
11        Modules in train mode
0         Modules in eval mode


Training for interval: 10


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


...

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


Validation: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_steps=1000` reached.
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


Predicting: |          | 0/? [00:00<?, ?it/s]

Reults for interval: 90
    unique_id         ds        LSTM
0        AAPL 2023-02-18  146.938995
1        AAPL 2023-02-19  147.419617
2        AAPL 2023-02-20  146.834778
3        AAPL 2023-02-21  147.261169
4        AAPL 2023-02-22  146.869385
..        ...        ...         ...
495      TSLA 2023-05-24  244.677765
496      TSLA 2023-05-25  246.336060
497      TSLA 2023-05-26  245.967316
498      TSLA 2023-05-27  246.911621
499      TSLA 2023-05-28  247.021851

[500 rows x 3 columns]


In [3]:
import numpy as np
import pandas as pd

input_sizes = [10,15,30,60,90]

# Define function for MAE calculation
def calculate_mae(y_true, y_pred):
    return np.mean(np.abs(np.array(y_true) - np.array(y_pred)))

# Define function for RMSE calculation
def calculate_rmse(y_true, y_pred):
    return np.sqrt(np.mean((np.array(y_true) - np.array(y_pred)) ** 2))

# Define function for RMAE calculation
def calculate_rmae(mae, avg_price):
    return (mae / avg_price) * 100 if avg_price != 0 else np.nan

# List to store per-stock results
detailed_results = []

# Iterate over each interval
for interval in input_sizes:
    df = pd.read_csv(f"LSTM_RESULTS_PER_STOCK/res_lstm_{interval}.csv")

    # Calculate RMSE, MAE, and RMAE for each unique_id (each stock separately)
    grouped_results = df.groupby("unique_id").apply(lambda group: {
        "unique_id": group["unique_id"].iloc[0],
        "Interval": interval,
        "MAE": calculate_mae(group["y"], group["LSTM"]),
        "RMSE": calculate_rmse(group["y"], group["LSTM"]),
        "RMAE": calculate_rmae(
            calculate_mae(group["y"], group["LSTM"]),
            np.mean(group["y"])  # Average price in the interval
        )
    }).tolist()  # Convert to list of dicts

    # Append per-stock results
    detailed_results.extend(grouped_results)

# Save results as DataFrame
detailed_results_df = pd.DataFrame(detailed_results)
detailed_results_df.to_csv("results_lstm.csv", index=False)


/var/folders/vw/fk4p6s510g7dfnhknk592c7r0000gn/T/ipykernel_67923/920193458.py:26: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  grouped_results = df.groupby("unique_id").apply(lambda group: {
/var/folders/vw/fk4p6s510g7dfnhknk592c7r0000gn/T/ipykernel_67923/920193458.py:26: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  grouped_results = df.groupby("unique_id").apply(lambda group: {
/var/folders/vw/fk4p6s510g